In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## important libraries

In [2]:
import seaborn as sns
base_dataset=sns.load_dataset("titanic")

In [3]:
base_dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Determine outliers columns

Outliers in input data can skew and mislead the training process of machine learning algorithms resulting in longer training times, less accurate models and ultimately poorer results.

In [4]:
outliers_columns=[]
for i in base_dataset.describe().columns:
    if base_dataset[i].nunique()>8:
        outliers_columns.append(i)
    

In [5]:
outliers_columns

['age', 'fare']

Detrmine the quartile values(threshold values), values outside quartile will be treated with median.

In [7]:
for i in outliers_columns:
    x=base_dataset[i].values
import numpy as np
qrt_1=np.quantile(x,0.25) #1st quartile
qrt_3=np.quantile(x,0.75) #3rd quartile
qrt_1
qrt_3
iqr=qrt_3-qrt_1     #interquartile range
utv=qrt_3+1.5*(iqr) #upper threshold value
ltv=qrt_1-1.5*(iqr) #lower threshold value
utv,ltv
outliers=[]
for i in x:
    if i < ltv or i>utv:
        outliers.append(i)
outliers=np.array(outliers)
outliers.min()     #min outliers value
outliers.max()     #max outliers value
outliers_treatment=[]  #outliers treated with median
for i in x:
    if i < ltv or i>utv:
        outliers_treatment.append(np.median(x))
    else:
        outliers_treatment.append(i)
base_dataset[i]=outliers_treatment
    


7.9104

31.0

(65.6344, -26.724)

66.6

512.3292

In [8]:
for i in outliers_columns: #values in outliers columns
    x=base_dataset[i].values
x

array([  7.25  ,  71.2833,   7.925 ,  53.1   ,   8.05  ,   8.4583,
        51.8625,  21.075 ,  11.1333,  30.0708,  16.7   ,  26.55  ,
         8.05  ,  31.275 ,   7.8542,  16.    ,  29.125 ,  13.    ,
        18.    ,   7.225 ,  26.    ,  13.    ,   8.0292,  35.5   ,
        21.075 ,  31.3875,   7.225 , 263.    ,   7.8792,   7.8958,
        27.7208, 146.5208,   7.75  ,  10.5   ,  82.1708,  52.    ,
         7.2292,   8.05  ,  18.    ,  11.2417,   9.475 ,  21.    ,
         7.8958,  41.5792,   7.8792,   8.05  ,  15.5   ,   7.75  ,
        21.6792,  17.8   ,  39.6875,   7.8   ,  76.7292,  26.    ,
        61.9792,  35.5   ,  10.5   ,   7.2292,  27.75  ,  46.9   ,
         7.2292,  80.    ,  83.475 ,  27.9   ,  27.7208,  15.2458,
        10.5   ,   8.1583,   7.925 ,   8.6625,  10.5   ,  46.9   ,
        73.5   ,  14.4542,  56.4958,   7.65  ,   7.8958,   8.05  ,
        29.    ,  12.475 ,   9.    ,   9.5   ,   7.7875,  47.1   ,
        10.5   ,  15.85  ,  34.375 ,   8.05  , 263.    ,   8.0

In [9]:
import numpy as np
qrt_1=np.quantile(x,0.25) #1st quartile value
qrt_3=np.quantile(x,0.75) #3rd quartile value

In [10]:
qrt_1

7.9104

In [11]:
qrt_3

31.0

#### Dummy variables

In [12]:
base_dataset['who'].unique()

array(['man', 'woman', 'child'], dtype=object)

To get the dummies of sex and embarked columns

In [14]:
import pandas as pd
pd.get_dummies(base_dataset[['sex','embarked']]).head()

,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,0,1,0,0,1
1,1,0,1,0,0
2,1,0,0,0,1
3,1,0,0,0,1
4,0,1,0,0,1


In [15]:
base_dataset['sex'].head()

0      male
1    female
2    female
3    female
4      male
Name: sex, dtype: object

### Label Encoders:

Encoding the characters to int values of particular columns

In [16]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit(base_dataset['who'])
x=le.transform(base_dataset['who'])
base_dataset['who']=x

LabelEncoder()

In [17]:
base_dataset['who'].head()

0    1
1    2
2    2
3    2
4    1
Name: who, dtype: int32

Encoding 'category' type columns

In [20]:
from sklearn.preprocessing import LabelEncoder
for i in base_dataset.describe(include='category').columns:
    le=LabelEncoder()
    base_dataset[i]=base_dataset[i].astype(str)
    le.fit(base_dataset[i])
    x=le.transform(base_dataset[i])
    base_dataset[i]=x


LabelEncoder()

LabelEncoder()

In [21]:
base_dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,7.75
0,0,3,male,22.0,1,0,7.2500,S,2,1,True,7,Southampton,no,False,7.2500
1,1,1,female,38.0,1,0,71.2833,C,0,2,False,2,Cherbourg,yes,False,14.4542
2,1,3,female,26.0,0,0,7.9250,S,2,2,False,7,Southampton,yes,True,7.9250
3,1,1,female,35.0,1,0,53.1000,S,0,2,False,2,Southampton,yes,False,53.1000
4,0,3,male,35.0,0,0,8.0500,S,2,1,True,7,Southampton,no,True,8.0500


In [22]:
base_dataset.dtypes

survived         int64
pclass           int64
sex             object
age            float64
sibsp            int64
parch            int64
fare           float64
embarked        object
class            int32
who              int32
adult_male        bool
deck             int32
embark_town     object
alive           object
alone             bool
7.75           float64
dtype: object

Encoding object type columns

In [23]:
from sklearn.preprocessing import LabelEncoder
for i in base_dataset.describe(include='object').columns:
    le=LabelEncoder()
    base_dataset[i]=base_dataset[i].astype(str)
    le.fit(base_dataset[i])
    x=le.transform(base_dataset[i])
    base_dataset[i]=x

LabelEncoder()

LabelEncoder()

LabelEncoder()

LabelEncoder()

In [25]:
base_dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,7.75
0,0,3,1,22.0,1,0,7.2500,2,2,1,True,7,2,0,False,7.2500
1,1,1,0,38.0,1,0,71.2833,0,0,2,False,2,0,1,False,14.4542
2,1,3,0,26.0,0,0,7.9250,2,2,2,False,7,2,1,True,7.9250
3,1,1,0,35.0,1,0,53.1000,2,0,2,False,2,2,1,False,53.1000
4,0,3,1,35.0,0,0,8.0500,2,2,1,True,7,2,0,True,8.0500


Encoding boolean type columns

In [26]:
from sklearn.preprocessing import LabelEncoder
for i in base_dataset.describe(include='bool').columns:
    le=LabelEncoder()
    base_dataset[i]=base_dataset[i].astype(str)
    le.fit(base_dataset[i])
    x=le.transform(base_dataset[i])
    base_dataset[i]=x

LabelEncoder()

LabelEncoder()

In [27]:
base_dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone,7.75
0,0,3,1,22.0,1,0,7.2500,2,2,1,1,7,2,0,0,7.2500
1,1,1,0,38.0,1,0,71.2833,0,0,2,0,2,0,1,0,14.4542
2,1,3,0,26.0,0,0,7.9250,2,2,2,0,7,2,1,1,7.9250
3,1,1,0,35.0,1,0,53.1000,2,0,2,0,2,2,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500,2,2,1,1,7,2,0,1,8.0500
